<a href="https://colab.research.google.com/github/coderanandmaurya/ANN/blob/main/new_word_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

# Sample text data
data = "Your text data goes here. It can be any length of text."

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for line in data.split('.'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Create predictors and label
X = input_sequences[:,:-1]
y = input_sequences[:,-1]
y = to_categorical(y, num_classes=total_words)


In [2]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.optimizers import Adam

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 6, 100)            1200      
                                                                 
 lstm (LSTM)                 (None, 6, 150)            150600    
                                                                 
 dropout (Dropout)           (None, 6, 150)            0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               100400    
                                                                 
 dense (Dense)               (None, 12)                1212      
                                                                 
Total params: 253412 (989.89 KB)
Trainable params: 253412 (989.89 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [3]:
history = model.fit(X, y, epochs=100, verbose=1)


Epoch 1/100
1/1 [==============================] - 6s 6s/step - loss: 2.4846 - accuracy: 0.1000
Epoch 2/100
1/1 [==============================] - 0s 20ms/step - loss: 2.4775 - accuracy: 0.4000
Epoch 3/100
1/1 [==============================] - 0s 19ms/step - loss: 2.4705 - accuracy: 0.2000
Epoch 4/100
1/1 [==============================] - 0s 19ms/step - loss: 2.4622 - accuracy: 0.2000
Epoch 5/100
1/1 [==============================] - 0s 19ms/step - loss: 2.4533 - accuracy: 0.2000
Epoch 6/100
1/1 [==============================] - 0s 21ms/step - loss: 2.4414 - accuracy: 0.2000
Epoch 7/100
1/1 [==============================] - 0s 20ms/step - loss: 2.4318 - accuracy: 0.2000
Epoch 8/100
1/1 [==============================] - 0s 21ms/step - loss: 2.4179 - accuracy: 0.2000
Epoch 9/100
1/1 [==============================] - 0s 21ms/step - loss: 2.4039 - accuracy: 0.2000
Epoch 10/100
1/1 [==============================] - 0s 20ms/step - loss: 2.3811 - accuracy: 0.2000
Epoch 11/100
1/1 [===

In [4]:
import numpy as np

def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)
    predicted_word = tokenizer.index_word[predicted_word_index[0]]
    return predicted_word

# Example usage
text = "Your input text"
next_word = predict_next_word(model, tokenizer, text, max_sequence_len)
print(next_word)


data
